This exercise will require you to pull some data from the Quandl API. Quandl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'XXXXXXXXXXXXXXXXXXXX'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [4]:
# First, import the relevant modules
import requests

In [43]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
quandl.ApiConfig.api_key = API_KEY
#data = quandl.get("FSE/AFX_X", start_date="2017-01-01", end_date="2017-01-30")
#print(data.info())

TKR = 'AFX_X'
url = "https://www.quandl.com/api/v3/datasets/FSE/%s/data.json?api_key=%s" % (TKR,API_KEY)
date_range = '&start_date=2017-01-01&end_date=2017-01-10'
results = requests.get(url+date_range).json()

In [44]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print (type(results))
print (results)

<class 'dict'>
{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2017-01-01', 'end_date': '2017-01-10', 'frequency': 'daily', 'data': [['2017-01-10', 34.8, 34.98, 34.46, 34.91, None, 43976.0, 1528055.0, None, None, None], ['2017-01-09', 35.29, 35.35, 34.43, 34.67, None, 62225.0, 2157182.0, None, None, None], ['2017-01-06', 34.91, 35.21, 34.91, 35.04, None, 27507.0, 964046.0, None, None, None], ['2017-01-05', 35.02, 35.2, 34.73, 35.06, None, 48412.0, 1692326.0, None, None, None], ['2017-01-04', 35.48, 35.51, 34.75, 35.19, None, 54408.0, 1906810.0, None, None, None], ['2017-01-03', 35.9, 35.93, 35.34, 35.48, None, 70618.0, 2515473.0, None, None, None], ['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [77]:
## Step 1
date_range = '&start_date=2017-01-01&end_date=2017-12-31'
results = requests.get(url+date_range)

## Step 2 results.json() already returns a Python dictionary.  Move the json() call down to here.
print(type(results.json()),'\n')
AFX = results.json()

## Step 3 Highest and Lowest openings
list_of_lists = AFX['dataset_data']['data']
#print(list_of_lists)
max_open = 0.0
min_open = 100000.0
for l in list_of_lists:   ## By inspection of the 'column_names', we see Open is the second item in each day's list
    Open = l[1]
    #print (l)
    try:
        if Open > max_open:
            max_open = Open
        if Open < min_open:
            min_open = Open
    except TypeError:
        pass
print ("Highest Opening price = {}, Lowest Opening price = {}".format(max_open, min_open))

<class 'dict'> 

Highest Opening price = 53.11, Lowest Opening price = 34.0


In [81]:
## Step 4 Biggest intraday swing
max_swing = 0
for l in list_of_lists:
    ## By inspection of column_names, High is third (#2) item and Low is the fourth (#3).
    try:
        swing = l[2] - l[3]
        if swing > max_swing:
            max_swing = swing
    except TypeError:
        pass
print ("Max intraday swing = {}".format(round(max_swing,2)))

Max intraday swing = 2.81


In [83]:
## Step 5 Biggest closing-to-closing swing
max_swing = 0
prev_close = None
for l in list_of_lists:
    ## By inspection of column_names, Close is fifth (#4) item.
    close = l[4]
    try:
        swing = close - prev_close
        if swing > max_swing:
            max_swing = swing
    except TypeError:
        swing = 0
    prev_close = close
    
print ("Max daily swing = {}".format(round(max_swing,2)))

Max daily swing = 2.56


In [97]:
volumes = [l[6] for l in list_of_lists if l[6] is not None]
print("Average daily trading volume = {}".format(round(sum(volumes) / len(volumes), 1)))

Average daily trading volume = 89124.3


In [99]:
volumes.sort()
mid = len(volumes) // 2
print("Median daily trading volume = {}".format(round((volumes[mid] + volumes[~mid]) / 2, 1)))

Median daily trading volume = 76286.0
